In [1]:
import torch
import torchvision
from torchvision import transforms
from MLProject2Dataset import *

In [2]:
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# batch_size = 4

# trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

# testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
# testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

In [10]:
dataloader = MLProject2Dataset('./data/dermoscopy_classification')
generator = torch.Generator().manual_seed(42)
train, validate, test = torch.utils.data.random_split(dataloader, [0.6, 0.1, 0.3], generator)

In [3]:
len(train), len(validate), len(test)

(6010, 1001, 3004)

In [4]:
image_transforms(10, 2)

Compose(
    Resize(size=(10, 2), interpolation=bilinear, max_size=None, antialias=warn)
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
)

In [8]:
# data_dir = './data/dermoscopy_classification/'
# metadata_fname = 'metadata.csv'


# files = pd.DataFrame({
#             'image_id': [Path(path).stem for path in glob.glob(f'{data_dir}/*/*')],
#             'path': [os.path.abspath(path) for path in glob.glob(f'{data_dir}/*/*')]
#         })

# metadata = pd.read_csv(data_dir + '/' + metadata_fname)
# metadata['dx'] = metadata['dx'].astype('category').cat.codes

# dataset = pd.merge(files, metadata[['dx', 'image_id']], on='image_id')


In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch import optim
import torchvision
import pandas as pd
import glob
import os
from pathlib import Path
import PIL

In [ ]:
# loss_fn = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.params(), lr=0.0001)

In [ ]:
def train(model: nn.Module, trainloader: DataLoader, valloader: DataLoader = None, 
          epochs: int = 10, optimizer: optim = None, loss: nn.modules.loss = None,
          device: str = 'cpu', print_period: int = 10) -> None:
    
    
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for batch, (X, y) in enumerate(trainloader, 0):
            # Move to device
            X, y = X.to(device), y.to(device)

            # Zero gradients
            optimizer.zero_grad()

            # Forward pass
            pred = model(X)
            current_loss = loss(pred, y)

            # Backpropagation
            current_loss.backward()

            # Parameter update
            optimizer.step()

            # Print

            
            running_loss += current_loss.item()

            
            if batch % print_period == print_period-1:
                avg_loss = running_loss / print_period

                
                model.eval()
                for batch, (X, y) in enumerate(trainloader, 0):
                
                    print(f'[Epoch: {epoch}, batch: {batch:5d}] loss: {avg_loss:.3f}')
                

In [ ]:
if seed != None:
            torch.manual_seed(seed)

        device = self.device

        loss_fn=self.loss_fn
        optimizer = self.optimizer       

        # print(f"Training on: {device}")

        epochs = epochs
        
        X_train, y_train, X_test, y_test = X_train.to(device), y_train.to(device), X_test.to(device), y_test.to(device)
        
        self.train()

        # for epoch in tqdm(range(epochs), desc='Training...'):
        for epoch in range(epochs):
            y_pred = self(X_train).to(device)

            loss = loss_fn(y_pred, y_train)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Calculate prediction accuracy
            acc = accuracy_fn(y_pred=torch.round(torch.sigmoid(y_pred)), y_true=y_train)

           
            ### Testing
            self.eval()
            with torch.inference_mode():
                y_pred_test = self(X_test)
                test_loss = loss_fn(y_pred_test, y_test)
                test_acc = accuracy_fn(y_pred=torch.round(torch.sigmoid(y_pred_test)), y_true=y_test)
          
            if epoch % int(epochs/10) == 0 or epoch == epochs-1:
                
                self.train_accuracy_progess.append({
                    'epoch': epoch,
                    'train_acc' : acc,
                    'test_acc': test_acc
                })

                if verbose: print(f'Epoch: {epoch:04d} | Loss: {loss:.4f}, Acc: {acc:.2f}% | Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')